In [1]:
# source source.me before launching jupyter
# export DWAVE_API_TOKEN=DEV-7dfad1c79ca4cac893a1541f6d04fbb3c2dced5c

In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=4
!nvidia-smi

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=4
Wed Sep 20 15:52:16 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.125.06   Driver Version: 525.125.06   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  On   | 00000000:01:00.0 Off |                    0 |
| N/A   56C    P0    75W / 275W |      3MiB / 81920MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA A100-SXM... 

In [2]:
import hydra
from hydra.utils import instantiate
from hydra import initialize, compose

In [3]:
import sys
import os
import getpass

os.chdir('/home/' + getpass.getuser() + '/Projects/CaloQVAE/')
sys.path.insert(1, '/home/' + getpass.getuser() + '/Projects/CaloQVAE/')

In [4]:
#external libraries
import os
import pickle
import datetime
import sys
import yaml
import json

import torch.optim as optim
import torch
from torch import nn
from torch.nn.functional import binary_cross_entropy_with_logits
torch.manual_seed(32)
import numpy as np
np.random.seed(32)
import matplotlib.pyplot as plt
import hydra
from hydra.utils import instantiate

from omegaconf import OmegaConf

# PyTorch imports
from torch import device, load, save
from torch.nn import DataParallel
from torch.cuda import is_available

# Add the path to the parent directory to augment search for module
sys.path.append(os.getcwd())
    
# Weights and Biases
import wandb

#self defined imports
from CaloQVAE import logging

[15:52:20.626] INFO   CaloQVAE                                          Willkommen!
[15:52:20.626] INFO   CaloQVAE                                          Loading configuration.


In [5]:
logger = logging.getLogger(__name__)

from data.dataManager import DataManager
from utils.plotting.plotProvider import PlotProvider
from engine.engine import Engine
from models.modelCreator import ModelCreator

from utils.plotting.HighLevelFeatures import HighLevelFeatures as HLF
HLF_1_photons = HLF('photon', filename='/raid/javier/Datasets/CaloVAE/data/atlas/binning_dataset_1_photons.xml', wandb=False)
HLF_1_pions = HLF('pion', filename='/raid/javier/Datasets/CaloVAE/data/atlas/binning_dataset_1_pions.xml', wandb=False)
HLF_1_electron = HLF('electron', filename='/raid/javier/Datasets/CaloVAE/data/atlas_dataset2and3/binning_dataset_2.xml', wandb=False)

[15:52:21.155] INFO   numexpr.utils                                     Note: detected 128 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
[15:52:21.156] INFO   numexpr.utils                                     Note: NumExpr detected 128 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
[15:52:21.156] INFO   numexpr.utils                                     NumExpr defaulting to 8 threads.
2023-09-20 15:52:21,578 dwave.cloud INFO  MainThread Log level for 'dwave.cloud' namespace set to 0
[15:52:21.578] INFO   dwave.cloud                                       Log level for 'dwave.cloud' namespace set to 0


In [6]:
hydra.core.global_hydra.GlobalHydra.instance().clear()
initialize(version_base=None, config_path="configs")

hydra.initialize()

In [25]:


# config=compose(config_name="config.yaml")
config=compose(config_name="config.yaml")
wandb.init(project="caloqvae", entity="qvae", config=config, mode='disabled')
modelCreator = ModelCreator(cfg=config)
dataMgr = DataManager(cfg=config)
#initialise data loaders
dataMgr.init_dataLoaders()
#run pre processing: get/set input dimensions and mean of train dataset
dataMgr.pre_processing()

if config.model.activation_fct.lower()=="relu":
    modelCreator.default_activation_fct=torch.nn.ReLU()
elif config.model.activation_fct.lower()=="tanh":
    modelCreator.default_activation_fct=torch.nn.Tanh()
else:
    logger.warning("Setting identity as default activation fct")
    modelCreator.default_activation_fct=torch.nn.Identity()

#instantiate the chosen model
#loads from file 
model=modelCreator.init_model(dataMgr=dataMgr)

#create the NN infrastructure
model.create_networks()

/raid/javier/anaconda3/lib/python3.11/site-packages/hydra/_internal/defaults_list.py:251: UserWarning: In 'config.yaml': Defaults list is missing `_self_`. See https://hydra.cc/docs/1.2/upgrades/1.0_to_1.1/default_composition_order for more information
  warnings.warn(msg, UserWarning)
[17:18:38.971] INFO   data.dataManager                                  Loading Data
[17:18:45.593] INFO   data.dataManager                                  <torch.utils.data.dataloader.DataLoader object at 0x7fea1c000450>: 80000 events, 625 batches
[17:18:45.594] INFO   data.dataManager                                  <torch.utils.data.dataloader.DataLoader object at 0x7fea1c002710>: 10000 events, 10 batches
[17:18:45.594] INFO   data.dataManager                                  <torch.utils.data.dataloader.DataLoader object at 0x7fea1eef28d0>: 10000 events, 10 batches
[17:18:47.712] INFO   models.modelCreator                               Initialising Model Type GumBoltCaloCRBM
2023-09-19 17:18:47,714

Initializing NetworkV3


In [26]:
#Not printing much useful info at the moment to avoid clutter. TODO optimise
model.print_model_info()
# for name, param in model.named_parameters():
#         print(name, param.requires_grad)

# Load the model on the GPU if applicable
# dev = None
# if (config.device == 'gpu') and config.gpu_list:
#     logger.info('Requesting GPUs. GPU list :' + str(config.gpu_list))
#     devids = ["cuda:{0}".format(x) for x in list(config.gpu_list)]
#     logger.info("Main GPU : " + devids[0])

#     if is_available():
#         print(devids[0])
#         dev = device(devids[0])
#         if len(devids) > 1:
#             logger.info(f"Using DataParallel on {devids}")
#             model = DataParallel(model, device_ids=list(config.gpu_list))
#         logger.info("CUDA available")
#     else:
#         dev = device('cpu')
#         logger.info("CUDA unavailable")
# else:
#     logger.info('Requested CPU or unable to use GPU. Setting CPU as device.')
#     dev = device('cpu')
dev = torch.device("cuda:0")

# Send the model to the selected device
# model.to(dev)
# Log metrics with wandb
wandb.watch(model)

# For some reason, need to use postional parameter cfg instead of named parameter
# with updated Hydra - used to work with named param but now is cfg=None 
engine=instantiate(config.engine, config)

#TODO for some reason hydra double instantiates the engine in a
#newer version if cfg=config is passed as an argument. This is a workaround.
#Find out why that is...
engine._config=config
#add dataMgr instance to engine namespace
engine.data_mgr=dataMgr
#add device instance to engine namespace
engine.device=dev    
#instantiate and register optimisation algorithm
engine.optimiser = torch.optim.Adam(model.parameters(),
                                    lr=config.engine.learning_rate)
#add the model instance to the engine namespace
engine.model = model
# add the modelCreator instance to engine namespace
engine.model_creator = modelCreator
engine.model = engine.model.to(dev)

[17:18:50.944] INFO   models.autoencoders.autoencoderbase               _dataset_mean: torch.Size([1, 6480])
[17:18:50.944] INFO   wandb                                             Watching
[17:18:50.947] INFO   engine.engineAtlas                                Setting up engine Atlas.
[17:18:50.947] INFO   engine.engineCaloV3                               Setting up engine Calo.
[17:18:50.947] INFO   engine.engine                                     Setting up default engine.


In [9]:
train_loader,test_loader,val_loader = engine.data_mgr.create_dataLoader()

[16:54:28.957] INFO   data.dataManager                                  <torch.utils.data.dataloader.DataLoader object at 0x7fec2d9f0990>: 80000 events, 625 batches
[16:54:28.957] INFO   data.dataManager                                  <torch.utils.data.dataloader.DataLoader object at 0x7feb43678250>: 10000 events, 10 batches
[16:54:28.958] INFO   data.dataManager                                  <torch.utils.data.dataloader.DataLoader object at 0x7fe9cf8a67d0>: 10000 events, 10 batches


In [36]:
from dwave.system import DWaveSampler
qpu_sampler = DWaveSampler(solver={"topology__type":"pegasus"})
qpu_nodes = qpu_sampler.nodelist
qpu_edges = qpu_sampler.edgelist

2023-09-20 17:12:08,300 dwave.cloud.client.base INFO  MainThread Using region metadata: [Region(code='na-west-1', name='North America', endpoint='https://na-west-1.cloud.dwavesys.com/sapi/v2/'), Region(code='eu-central-1', name='Europe', endpoint='https://eu-central-1.cloud.dwavesys.com/sapi/v2/')]
[17:12:08.300] INFO   dwave.cloud.client.base                           Using region metadata: [Region(code='na-west-1', name='North America', endpoint='https://na-west-1.cloud.dwavesys.com/sapi/v2/'), Region(code='eu-central-1', name='Europe', endpoint='https://eu-central-1.cloud.dwavesys.com/sapi/v2/')]
2023-09-20 17:12:08,306 dwave.cloud.client.base INFO  MainThread Fetching definitions of all available solvers
[17:12:08.306] INFO   dwave.cloud.client.base                           Fetching definitions of all available solvers
2023-09-20 17:12:09,656 dwave.cloud.client.base INFO  MainThread Received solver data for 6 solver(s).
[17:12:09.656] INFO   dwave.cloud.client.base                

In [37]:
# plt.plot(qpu_sampler.nodelist)
node_list = dict()
node_list["less15"] = []
node_list["greater15"] = []
node_list["eq15"] = []
for key in qpu_sampler.adjacency.keys():
    if len(qpu_sampler.adjacency[key]) < 15:
        node_list["less15"].append(key)
    elif len(qpu_sampler.adjacency[key]) == 15:
        node_list["eq15"].append(key)
    else:
        node_list["greater15"].append(key)

In [39]:
len(node_list["eq15"])
# len(node_list["less15"])

4184

In [12]:
sampler = DWaveSampler()

qubit_a = sampler.nodelist[0]
qubit_b = next(iter(sampler.adjacency[qubit_a]))
sampleset = sampler.sample_ising({qubit_a: -1, qubit_b: 1},{}, num_reads=100)
sampleset.first.sample[qubit_a] == 1 and sampleset.first.sample[qubit_b] == -1

2023-09-20 15:59:03,962 dwave.cloud.client.base INFO  MainThread Using region metadata: [Region(code='na-west-1', name='North America', endpoint='https://na-west-1.cloud.dwavesys.com/sapi/v2/'), Region(code='eu-central-1', name='Europe', endpoint='https://eu-central-1.cloud.dwavesys.com/sapi/v2/')]
[15:59:03.962] INFO   dwave.cloud.client.base                           Using region metadata: [Region(code='na-west-1', name='North America', endpoint='https://na-west-1.cloud.dwavesys.com/sapi/v2/'), Region(code='eu-central-1', name='Europe', endpoint='https://eu-central-1.cloud.dwavesys.com/sapi/v2/')]
2023-09-20 15:59:03,968 dwave.cloud.client.base INFO  MainThread Fetching definitions of all available solvers
[15:59:03.968] INFO   dwave.cloud.client.base                           Fetching definitions of all available solvers
2023-09-20 15:59:05,132 dwave.cloud.client.base INFO  MainThread Received solver data for 6 solver(s).
[15:59:05.132] INFO   dwave.cloud.client.base                

True

In [15]:
qubit_b

2985

In [16]:
sampler.adjacency[qubit_a]

{31, 45, 2940, 2955, 2970, 2985}

In [17]:
sampleset

SampleSet(rec.array([([ 1, -1], -2., 100)],
          dtype=[('sample', 'i1', (2,)), ('energy', '<f8'), ('num_occurrences', '<i8')]), Variables([30, 2985]), {'timing': {'qpu_sampling_time': 8260.0, 'qpu_anneal_time_per_sample': 20.0, 'qpu_readout_time_per_sample': 42.06, 'qpu_access_time': 24021.17, 'qpu_access_overhead_time': 469.83, 'qpu_programming_time': 15761.17, 'qpu_delay_time_per_sample': 20.54, 'total_post_processing_time': 251.0, 'post_processing_overhead_time': 251.0}, 'problem_id': '8da59306-2495-45ac-aa3f-0c862789d7c7'}, 'SPIN')

In [20]:
import math

In [35]:
_CELL_SIDE_QUBITS = 4
_MAX_ROW_COLS = 16

n_visible = 5
n_hidden = 5

# self._n_visible=n_visible
# self._n_hidden=n_hidden

require_grad=True

n_cells = max(math.ceil(n_visible/_CELL_SIDE_QUBITS), math.ceil(n_hidden/_CELL_SIDE_QUBITS))
n_rows = math.ceil(math.sqrt(n_cells))
n_cols = n_rows

assert n_cols<=_MAX_ROW_COLS

visible_qubit_idxs = []
hidden_qubit_idxs = []

#         qpu_sampler = DWaveSampler(solver={"topology__type":"chimera", "chip_id":"DW_2000Q_6"})
qpu_sampler = DWaveSampler(solver={"topology__type":"chimera"})
qpu_nodes = qpu_sampler.nodelist
qpu_edges = qpu_sampler.edgelist

## The n_rows+1 and n_cols+1 ensure that we get correct number of visible and hidden qubits
for row in range(n_rows+1):    
    for col in range(n_cols+1):
        for n in range(_CELL_SIDE_QUBITS):
            if (len(visible_qubit_idxs) < n_visible) or (len(hidden_qubit_idxs) < n_hidden):
                idx = 8*row + 8*col*_MAX_ROW_COLS + n
                # Even cell
                if (row+col)%2 == 0:
                    if idx in qpu_nodes:
                        visible_qubit_idxs.append(idx)
                    if idx+4 in qpu_nodes:
                        hidden_qubit_idxs.append(idx+4)
                # Odd cell
                else:
                    if idx in qpu_nodes:
                        hidden_qubit_idxs.append(idx)
                    if idx+4 in qpu_nodes:
                        visible_qubit_idxs.append(idx+4)

# Remove extra nodes from the qubit idxs lists
visible_qubit_idxs = visible_qubit_idxs[:n_visible]
hidden_qubit_idxs = hidden_qubit_idxs[:n_hidden]

2023-09-20 16:28:20,398 dwave.cloud.client.base INFO  MainThread Using region metadata: [Region(code='na-west-1', name='North America', endpoint='https://na-west-1.cloud.dwavesys.com/sapi/v2/'), Region(code='eu-central-1', name='Europe', endpoint='https://eu-central-1.cloud.dwavesys.com/sapi/v2/')]
[16:28:20.398] INFO   dwave.cloud.client.base                           Using region metadata: [Region(code='na-west-1', name='North America', endpoint='https://na-west-1.cloud.dwavesys.com/sapi/v2/'), Region(code='eu-central-1', name='Europe', endpoint='https://eu-central-1.cloud.dwavesys.com/sapi/v2/')]
2023-09-20 16:28:20,405 dwave.cloud.client.base INFO  MainThread Fetching definitions of all available solvers
[16:28:20.405] INFO   dwave.cloud.client.base                           Fetching definitions of all available solvers
2023-09-20 16:28:21,479 dwave.cloud.client.base INFO  MainThread Received solver data for 6 solver(s).
[16:28:21.479] INFO   dwave.cloud.client.base                

SolverNotFoundError: Solver with the requested features not available

In [34]:
# print(n_cells, n_rows, n_cols)
visible_qubit_idxs

[132, 133, 134, 135, 256]

In [28]:
hidden_qubit_idxs

[128, 129, 130, 131, 260]

In [30]:
sampler.adjacency[132]

{131, 133, 147, 5130, 5145, 5220, 5235, 5250, 5265, 5280, 5295}

In [33]:
qpu_sampler.edgelist[132]

(53, 4410)